In [2]:
import pandas as pd
import numpy as np

In [5]:
df_finance = pd.read_csv("../feature_extraction/market_features.csv").set_index("Date")
tickers = pd.read_csv("../feature_extraction/valid_tickers.csv")

tickers = tickers["ticker"].tolist()

df_finance.columns = pd.MultiIndex.from_tuples(
    [(col.split("_")[0], "_".join(col.split("_")[1:])) for col in df_finance.columns]
)

train_features = ['Close', 'High', 'Low', 'Open', 'RSI_14', 'Volatility_21', 'Volume', 'Volume_Z', 'Sentiment',]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_all, y_all = [], []
window_prediction = 30

for ticker in tickers:
  try:
    df_ticker_finance = df_finance[ticker]
    df_train = df_ticker_finance[train_features].copy()
    #add wasserstein distance for each ticker

    df_train = df_train.merge(df_finance["Wasserstein"].loc[df_train.index], left_index=True, right_index=True)
    df_train.loc[:, "Target"] = df_ticker_finance["Log_Return"] # predict log return

    df_train[train_features] = scaler.fit_transform(df_train[train_features])
  except Exception as e:
    print(f"Error{ticker}: {e}")
    continue
  
  X, y = [], []

  for i in range(len(df_train) - window_prediction):
    X.append(df_train[train_features].iloc[i:i+window_prediction].values)
    y.append(df_train['Target'].iloc[i+window_prediction])
  X, y = np.array(X), np.array(y)
  X_all.append(X)
  y_all.append(y)

EMN
CTAS
V
BLK
GE
COP
BA
GDDY
WY
NWSA
SWK
DD
AKAM
DVN
EFX
ED
SPGI
ADBE
USB
KIM
ON
RJF
AME
PRU
CLX
PLD
FIS
PFG
BRO
IRM
SHW
CAT
STZ
BR
UNH
PAYC
CINF
KEYS
AZO
PGR
A
DUK
SYY
VLO
FI
CBRE
MMM
Error for MMM: "['Sentiment'] not in index"
GOOGL
LVS
LII
IVZ
BSX
ULTA
MSCI
MAS
APO
AMZN
LULU
ROL
WRB
INTC
LW
VTRS
ETR
DXCM
EW
TROW
LUV
JNJ
NVR
ATO
CMI
AMP
ITW
HUM
KDP
ALB
TKO
JBL
HAL
ROP
GOOG
MTB
CRL
NTRS
NUE
PANW
BIIB
EXC
RTX
EL
BK
TMUS
MLM
COF
ORCL
NXPI
EA
BX
CDNS
UAL
MTD
UNP
INVH
CAG
EMR
REGN
MKC
LMT
WAB
META
AIG
IBM
Error for IBM: "['Sentiment'] not in index"
PM
BMY
PAYX
KLAC
NOW
NSC
TSN
F
TSLA
NVDA
BAC
FITB
GLW
VRSK
FSLR
NFLX
ANET
ENPH
PSX
CB
DVA
AMCR
PG
ORLY
CDW
DTE
CNP
APTV
BKR
CHRW
CPB
KHC
MET
ADSK
TFC
AEE
HIG
WST
EBAY
MOS
EXPD
MHK
TDG
CF
FTV
IPG
CL
IDXX
BDX
VST
ETN
TRV
SW
IEX
Error for IEX: "['Sentiment'] not in index"
VRSN
HOLX
FANG
SNA
HRL
LIN
POOL
STE
GWW
XYL
EQIX
MTCH
LRCX
CVX
J
OXY
AWK
ERIE
AIZ
SPG
IQV
INCY
LYV
RSG
WYNN
WSM
VICI
JKHY
CHTR
SJM
GEN
ZBH
NI
CHD
WMT
LOW
DLR
CMG
CPRT
UHS
CRM
L


In [20]:
X_all = np.array(X_all)
y_all = np.array(y_all)

X_all = X_all.reshape(-1, X_all.shape[2], X_all.shape[3])
y_all = y_all.reshape(-1)

In [21]:
np.save("X_all.npy", X_all)
np.save("y_all.npy", y_all)